# LCEL(LangChain Expression Language)

- LangChain에서 복잡한 작업의 흐름을 간단하게 만들고 관리할 수 있는 도구
- Chain(체인): 작업의 흐름을 연결하는 것.
- LCEL를 이용하면 여러 줄로 표현해야 하는 작업 단계를 읽기 쉽게 축약할 수 있고, 스트리밍 출력 등 여러 작업을 병렬로 처리할 수 있음.

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser  # 답변 출력
from langchain_core.prompts import ChatPromptTemplate  # 프롬프트 템플릿

# 출력 파서(Output Parser)

- 출력 파서: LLM이 반환하는 결과에서 원하는 형식의 데이터를 추출하는 도구.
  - 텍스트, JSON, 숫자 등 특정 형식을 처리할 수 있음.
- `StrOutputParser`: LLM이 반환하는 결과에서 텍스트(content)만 추출하는 객체.
  - `model.invoke()` 메서드의 반환 값을 `parser.invoke()` 메서드에 아규먼트로 전달하면, 답변 텍스트(content)만 추출할 수 있음.

In [2]:
load_dotenv()  # api_key를 OS 환경 변수에 로딩.

True

In [3]:
model = ChatOpenAI(model='gpt-4o-mini')  # LLM 모델 선택

In [4]:
messages = [
    SystemMessage('너는 미녀와 야수의 미녀 역할이야. 캐릭터에 맞게 대화해줘.'),
    HumanMessage('안녕하세요, 저는 가스통입니다. 저랑 저녁이나 같이 먹을까요?'),
]

In [5]:
ai_message = model.invoke(input=messages)

In [6]:
print(ai_message)

content='안녕하세요, 가스통. 당신의 제안은 고맙지만, 저는 다른 생각이 있어요. 제 마음은 다른 곳에 있어요. 하지만 당신의 친구로 지내는 건 언제나 환영이에요. 저녁 시간은 친구들과 나누는 것도 좋지요.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 55, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CEqRzJVCOyWtidAcxeXepvWznw3Za', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--5ff09212-b912-4f7f-bc07-dcf2405d1a28-0' usage_metadata={'input_tokens': 55, 'output_tokens': 63, 'total_tokens': 118, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


`AIMessage` 객체에서 content를 직접 추출해서 화면에 출력.

In [7]:
print(ai_message.content)

안녕하세요, 가스통. 당신의 제안은 고맙지만, 저는 다른 생각이 있어요. 제 마음은 다른 곳에 있어요. 하지만 당신의 친구로 지내는 건 언제나 환영이에요. 저녁 시간은 친구들과 나누는 것도 좋지요.


In [8]:
parser = StrOutputParser()  # Parser 객체 생성

In [9]:
parser.invoke(ai_message)  # Parser에게 AIMessage를 분석해서 텍스트만 추출하고 리턴.

'안녕하세요, 가스통. 당신의 제안은 고맙지만, 저는 다른 생각이 있어요. 제 마음은 다른 곳에 있어요. 하지만 당신의 친구로 지내는 건 언제나 환영이에요. 저녁 시간은 친구들과 나누는 것도 좋지요.'

체인 연산자(chain operator, `|`)

In [10]:
chain = model | parser

In [11]:
result = chain.invoke(input=messages)

In [12]:
print(result)

안녕하세요, 가스통. 정말 감사하지만, 저는 저녁을 함께할 수 없어요. 대신 저와 다르게 다른 사람들을 존중하고 배려할 수 있다면, 좋은 친구가 될 수 있을 거예요. 제가 찾고 있는 것은 진정한 사랑이니까요. 당신은 그걸 이해할 수 있을까요?


In [13]:
result = chain.stream(input=messages)
for msg in result:
    print(msg, end='')

안녕하세요, 가스통. 당신의 제안은 정말 고맙지만, 저는 당신과 저녁을 먹을 수는 없어요. 저는 당신의 그 매력적인 외모 대신, 진정한 사랑과 따뜻함을 가진 사람을 기다리고 있어요. 당신의 마음도 사랑으로 가득하길 바라요.

# Prompt Template

AI 모델을 호출(invoke)할 때 전달하는 메시지 형식이 매번 비슷할 때, 미리 템플릿을 작성해 두고 필요한 부분만 수정하면서 실행하는 방법.

In [4]:
system_prompt = '너는 {story}에 나오는 {assistant_character}야. 역할에 맞게 사용자와 대화해줘.'
human_prompt = '안녕? 나는 {user_character}야. 오늘 시간 있으면 {activity} 같이 할까?'
prompt_template = ChatPromptTemplate([
    ('system', system_prompt),
    ('human', human_prompt),
])
#> argument: ('role', prompt_message) 아이템으로 갖는 리스트

In [5]:
# 프롬프트 템플릿의 {variable}의 값을 채움.
# -> ChatPromptTemplate을 메시지들(SystemMessage, HumanMessage)의 리스트(list)로 변환.
messages = prompt_template.invoke({
    'story': '미녀와 야수',
    'assistant_character': '벨',
    'user_character': '가스통',
    'activity': '저녁',
})

In [6]:
print(messages)  #> [SystemMessage, HumanMessage]

messages=[SystemMessage(content='너는 미녀와 야수에 나오는 벨야. 역할에 맞게 사용자와 대화해줘.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕? 나는 가스통야. 오늘 시간 있으면 저녁 같이 할까?', additional_kwargs={}, response_metadata={})]


In [7]:
response = model.invoke(input=messages)

In [8]:
print(response.content)

안녕, 가스통. 네 제안은 고마워, 하지만 나는 다른 일로 바쁘단다. 나는 좀 더 깊은 사랑을 찾고 싶어. 나와 함께 해 줄 수 있는 사람은 이렇게 가벼운 데이트로 만족하지 않아. 너도 좀 더 진지한 생각을 해보면 좋겠어.


In [9]:
type(response)

langchain_core.messages.ai.AIMessage

In [10]:
response.pretty_print()

================================== Ai Message ==================================

안녕, 가스통. 네 제안은 고마워, 하지만 나는 다른 일로 바쁘단다. 나는 좀 더 깊은 사랑을 찾고 싶어. 나와 함께 해 줄 수 있는 사람은 이렇게 가벼운 데이트로 만족하지 않아. 너도 좀 더 진지한 생각을 해보면 좋겠어.


In [11]:
messages = prompt_template.invoke({
    'story': '미녀와 야수',
    'assistant_character': '벨',
    'user_character': '야수',
    'activity': '저녁',
})

In [12]:
response = model.invoke(input=messages)

In [13]:
response.pretty_print()

================================== Ai Message ==================================

안녕하세요, 야수님! 저녁을 함께 하는 건 정말 좋은 생각이에요. 멋진 저녁을 준비해주실 거죠? 어떤 음식을 준비할지 궁금해요.


## chain을 사용한 프롬프트 템플릿

In [14]:
chain = prompt_template | model

In [15]:
response = chain.invoke({
    'story': '배트맨',
    'assistant_character': '조커',
    'user_character': '할리 퀸',
    'activity': '은행 털기',
})

In [16]:
response.pretty_print()

================================== Ai Message ==================================

오, 할리! 내 사랑스러운 광기 어린 파트너! 은행 털기라니, 그건 굉장히 짜릿한 제안이야! 하지만 은행 털기 전에 좀 더 흥미로운 계획을 세워야겠지? 최악의 상황에서 얼마나 많은 혼란을 만들 수 있을지 생각해보는 건 어떨까? 자, 함께 즐거운 계획을 세워볼까? 왜냐하면 혼돈이 우리를 부르니까! Ha Ha Ha!
